# Virtually metabolize GNPS annotations and prepare for Network Annotation Propagation or SIRIUS

Made by Louis-Felix Nothias (UC San Diego), louisfelix.nothias@gmail.com. Started in 2018 and improved in May 2021.

This notebook downloads results of spectral annotations from classical or feature-based molecular networking job from GNPS [[http://gnps.ucsd.edu](http://gnps.ucsd.edu)] and generate virtual metabolites either with SyGMa or BioTransformer. The resulting candidates can be used for [Network Annotation Propagation](https://ccms-ucsd.github.io/GNPSDocumentation/nap/) on GNPS or with [SIRIUS](https://boecker-lab.github.io/docs.sirius.github.io/install/).

> Start by running the cell below to initiate the libraries.

In [1]:
import pandas as pd
import numpy as np
import sygma
from rdkit import Chem
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.ERROR)
import os
import sys
from convert_structures import *
from run_virtual_metabolization import *

RDKit WARNING: [14:09:58] Enabling RDKit 2019.09.3 jupyter extensions


## Mandatory - Download annotation from the GNPS job
 
> Replace the job ID from the GNPS molecular networking job in the URL in the cell below (line 3). We support both classical molecular networking and feature-based molecular networking (FBMN) jobs.

You can try the classical MN job from that paper https://pubs.acs.org/doi/10.1021/acs.analchem.8b05854 with the ID `bbee697a63b1400ea585410fafc95723`. 

An other test job for feature-based molecular networking (FBMN) is `e78a8c8f429a46fcb24f3b34d69aff25`.

In [2]:
!rm -r all_annotations.zip all_annotations/

# Replace the job ID in the line below <<< ====
!curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=bbee697a63b1400ea585410fafc95723&view=view_all_annotations_DB' -o all_annotations.zip

! unzip -q -d all_annotations/ all_annotations.zip
print('==================')
print('Downloading the GNPS job results')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.5M    0 29.5M    0     0  3067k      0 --:--:--  0:00:09 --:--:-- 3202k


In [112]:
try :
    path = [x for x in os.listdir('all_annotations/result_specnets_DB')][0]
    df_annotations = pd.read_csv('all_annotations/result_specnets_DB/'+path, sep='\t')
    print('==================')
    print('Classical MN job detected')
    print('==================')
    print('Number of annotations = '+str(df_annotations.shape[0]))
except: 
    print('==================')
    path = [x for x in os.listdir('all_annotations/DB_result')][0]
    df_annotations = pd.read_csv('all_annotations/DB_result/'+path, sep='\t')
    print('==================')
    print('FBMN job detected')
    print('==================')
    print('   Number of annotations in job = '+str(df_annotations.shape[0]))

Classical MN job detected
Number of annotations = 200


## Mandatory - Consolidating structures identifier

> Run the cell below to have a complete set of Smiles and InChI for the annotations.

In [113]:
consolidate_and_convert_structures(df_annotations,'GNPS_LIB_', smiles='Smiles', \
                                   inchi='INCHI')

Both SMILES and InChI were inputted
Converting SMILES to mol object
Succesfully converted to mol object: 110
Exception to the parsing: 0
Not available: 90
Converting INCHI to mol object
Succesfully converted to mol object: 104
Exception to the parsing: 0
Not available: 96
Consolidating the lists
Total mol object from the list 1 = 110
Mol object consolidated from list 2 = 12
Consolidated structures = 122
Converting mol objects to SMILES iso
Converting mol objects to SMILES
Converting mol objects to InChI
Converting mol objects to InChIKey
End


### Optional - Filter annotations based on compound name (skip otherwise)

If you don't want to apply this filter, skipped the following optional cells

##### Optional - Display compound name

In [118]:
list_compounds = set(df_annotations['Compound_Name'])
for item in sorted(list_compounds):
    print('\''+item+'\',')
    print('')
del list_compounds

'(+)-Catechin',

'(-)-Catechin',

'(-)-Secoisolariciresinol',

'(.+/-.)-8-Hydroxy-5Z,9E,11Z,14Z,17Z-eicosapentaenoic acid',

'.alpha.-Cyclodextrin',

'.alpha.-Hexylcinnamaldehyde',

'.alpha.-Ionone',

'2-(Cyclohexylamino)ethanesulfonic acid',

'3,4'-Dimethoxy-2-hydroxychalcone',

'3-(2-Hydroxyphenyl)propionic acid',

'3-Methoxycinnamic acid',

'4-(4-Aminophenoxy)aniline',

'4-Hydroxy-4'-methyldiphenylamine',

'7-Oxocholesterol',

'B10A30 Faulkner legacy library looks like sterol or lipid needs to be verified',

'Benzalkonium chloride (C12)',

'Betulinic acid',

'Betulinic acid methyl ester',

'CocamidopropylBetaine',

'Conjugated linoleic Acid (10E,12Z)',

'Dimethyldioctadecylammonium cation',

'Dioctyl phthalate',

'Escitalopram Oxalate',

'Ethyl 3-hydroxybenzoate',

'Glycerol tricaprate',

'Glycerol tricaprylate',

'Isobergaptene',

'Isoxadifen-ethyl',

'Lauric acid diethanolamide',

'Lyso-PC(16:0)',

'MLS000028461-01!URSODEOXYCHOLIC ACID',

'MLS001332387-01!Chlorhexidine55-56-1',

'

##### Optional - Select compound name to keep

Replace the compound names in the list `compound_name_to_keep`


In [107]:
compound_name_to_keep = ['Isobergaptene','Isoxadifen-ethyl','Lauric acid diethanolamide','Lyso-PC(16:0)',
                         'MLS000028461-01!URSODEOXYCHOLIC ACID','MLS001332387-01!Chlorhexidine55-56-1','MLS002154090-01!Proguanil hydrochloride637-32-1'
                        ]
df_annotations = df_annotations[df_annotations.Compound_Name.isin(compound_name_to_keep)]
print('Number of annotations after compound name filtering = '+str(df_annotations.shape[0]))

Number of annotations after compound name filtering = 0


### Optional - Filter annotations based on tags (skip otherwise)

If you don't want to apply this filter, skipped the following cells

##### Optional - Display tags in the annotations

In [6]:
set(df_annotations['tags'])

{' ', 'Antibiotic[Drug Class]'}

##### Optional - Select tags to keep

Specify the tags in the list `tags_to_keep`

In [137]:
tags_to_keep = ['Antibiotic[Drug Class]','other_tag_here']
df_annotations = df_annotations[df_annotations.tags.isin(tags_to_keep)]
print('Number of annotations after compound name filtering = '+str(df_annotations.shape[0]))

Number of annotations after compound name filtering = 1


## Mandatory - Choose between planar or stereochemical SMILES

### [RECOMMENDED] Use the planar SMILES for in silico metabolization (no stereochemistry specified)

Run the cell below to use planar isomers and ignore the cell after.

In [5]:
df_annotations = df_annotations[df_annotations.GNPS_LIB_Consol_SMILES_iso.str.contains('nan') == False]

df_annotations = df_annotations.sort_values(by=['MQScore'], ascending=False)
df_annotations = df_annotations.drop_duplicates(subset='GNPS_LIB_Consol_SMILES_iso', keep='first')

list_compound_name = list(df_annotations['Compound_Name'])
list_smiles = list(df_annotations['GNPS_LIB_Consol_SMILES_iso'])
print('Number of unique Compound Name = '+str(len(list_compound_name)))
print('Number of unique SMILES = '+str(len(list_smiles)))

Number of unique Compound Name = 57
Number of unique SMILES = 57


### [Skip the cell below] or instead use the stereochemical SMILES for virtual metabolization [Skip otherwise]
Run the cell below to use all the stereoisomers detected.

In [36]:
df_annotations = df_annotations[df_annotations.GNPS_LIB_Consol_SMILES.str.contains('nan') == False]

df_annotations = df_annotations.sort_values(by=['MQScore'], ascending=False)
df_annotations = df_annotations.drop_duplicates(subset='GNPS_LIB_Consol_SMILES', keep='first')

list_compound_name = list(df_annotations['Compound_Name'])
list_smiles = list(df_annotations['GNPS_LIB_Consol_SMILES'])
print('Number of unique Compound Name = '+str(len(list_compound_name)))
print('Number of unique SMILES = '+str(len(list_smiles)))

Number of unique Compound Name = 57
Number of unique SMILES = 57


# Mandatory - Choose between SyGMa (A) or BioTransformer (B) for virtual metabolization

#### A - SyGMa generates specifically human biotransformation of phase 1 and/or 2. 
It takes generally couple minutes to compute. More informations from the paper (https://doi.org/10.1002/cmdc.200700312).

#### B - BioTransformer generates biotransformation in mammals, their gut microbiota, as well as the soil/aquatic microbiota. 
It takes more time to compute. More information from the paper ([https://doi.org/10.1186/s13321-018-0324-5](https://doi.org/10.1186/s13321-018-0324-5)).

# A - Virtual metabolization with SyGMa

SyGMa is a python library for the Systematic Generation of potential Metabolites. See [SyGMa: combining expert knowledge and empirical scoring in the prediction of metabolites](https://doi.org/10.1002/cmdc.200700312) and [https://github.com/3D-e-Chem/sygma](https://github.com/3D-e-Chem/sygma).

Please cite their work:
Ridder, L., & Wagener, M. (2008) [SyGMa: combining expert knowledge and empirical scoring in the prediction of metabolites](https://doi.org/10.1002/cmdc.200700312). ChemMedChem, 3(5), 821-832.


### IMPORTANT
> Define the ruleset and the number of phase 1/2 reaction cyles to apply in the SyGMA scenario. For example 2 cycles for phase 1 `phase_1_cycle = 2`. Using a value > 1 will be slow.

> Define the maximum number of SyGMa candidates outputted (consider the number of reaction cycles). Suggested value `top_sygma_candidates = 15`

> Run SyGMa.

In [95]:
# Define the number of metabolization cycles (1-3). If the number of cycle is more than 1, it can be slow.
phase_1_cycle = 2
phase_2_cycle = 1

# Batch size 
batch_size = 50/(phase_1_cycle)/(math.exp(phase_2_cycle))

print(batch_size)
                  
#Top metabolites predicted by SyGMa to output (ranked by highest score)
top_sygma_candidates = 15

9.196986029286059


### Run the cell below for running SyGMa

In [96]:
run_sygma_batch(list_smiles, list_compound_name, phase_1_cycle, phase_2_cycle, top_sygma_candidates,'results_vm-NAP_SyGMa.tsv', int(batch_size))

=== Starting SyGMa computation ===
Number of compounds = 57
Batch_size = 9
If you are running many compounds or cycles, and maxing out RAM memory available, you can decreased the batch size. Otherwise the value can be increased for faster computation.
Please wait
Batch 1 was compeleted
Batch 2 was compeleted
Batch 3 was compeleted
Batch 4 was compeleted
Batch 5 was compeleted
Batch 6 was compeleted
Batch 7 was compeleted
Number of SyGMA candidates = 855
Number of unique SyGMA candidates = 813
===== COMPLETED =====


When completed, download the full SyGMa results in the left side panel->
['results_vm-NAP_SyGMa.tsv'](./results_vm-NAP_SyGMa.tsv).

## Export the results for NAP
See the documentation for custom database in [NAP](https://ccms-ucsd.github.io/GNPSDocumentation/nap/#structure-database) and how to run NAP on GNPS [https://ccms-ucsd.github.io/GNPSDocumentation/nap/#structure-database](https://ccms-ucsd.github.io/GNPSDocumentation/nap/#structure-database).

In [97]:
export_for_NAP('results_vm-NAP_SyGMa.tsv')

Download the results for NAP in the left side panel->
['results_vm-NAP_SyGMa_NAP.tsv'](./results_vm-NAP_SyGMa_NAP.tsv).

## Export the results for SIRIUS

See the documentation to generate the SIRIUS [custom database here](https://boecker-lab.github.io/docs.sirius.github.io/cli-standalone/#custom-database-tool).

In [98]:
export_for_SIRIUS('results_vm-NAP_SyGMa.tsv')

Download the results for SIRIUS in the left side panel->
['results_vm-NAP_SyGMa_SIRIUS.tsv'](./results_vm-NAP_SyGMa_SIRIUS.tsv?download=1).

# B - Virtual metabolization with BioTransformer (slow)

BioTransformer is a software tool that predicts small molecule metabolism in mammals, their gut microbiota, as well as the soil/aquatic microbiota. BioTransformer also assists scientists in metabolite identification, based on the metabolism prediction. More information from the paper [[https://doi.org/10.1186/s13321-018-0324-5](https://doi.org/10.1186/s13321-018-0324-5)] and [[https://bitbucket.org/djoumbou/biotransformerjar/src/master/](https://bitbucket.org/djoumbou/biotransformerjar/src/master/)].

### Citation

Djoumbou-Feunang, Y., Fiamoncini, J., Gil-de-la-Fuente, A. et al. [BioTransformer: a comprehensive computational tool for small molecule metabolism prediction and metabolite identification.](https://doi.org/10.1186/s13321-018-0324-5) J Cheminform 11, 2 (2019).

### Install BioTransformer
It requires `curl` and `java`.

In [99]:
!java -version
!rm -r biotransformer.zip biotransformer/
!curl https://bitbucket.org/djoumbou/biotransformerjar/get/f47aa4e3c0da.zip -o biotransformer.zip
!unzip -q -d biotransformer biotransformer.zip
!cp -r biotransformer/djoumbou-biotransformerjar-f47aa4e3c0da/. .
!rm -r biotransformer.zip biotransformer/

java version "1.8.0_281"
Java(TM) SE Runtime Environment (build 1.8.0_281-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.281-b09, mixed mode)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 68.9M  100 68.9M    0     0  15.0M      0  0:00:04  0:00:04 --:--:-- 17.6M


#### Specify the parameters of BioTransformer

`type_of_biotransformation` : -b,--btType <BioTransformer Option> The type of description: Type of biotransformer - EC-based (ecbased), CYP450 (cyp450), Phase II (phaseII), Human gut microbial (hgut), human super transformer* (superbio, or allHuman), Environmental microbial (envimicro).

`number_of_steps`  -s,--nsteps <Number of steps> The number of steps for the prediction. This option can be set by the user for the EC-based, CYP450, Phase II, and Environmental microbial biotransformers. The default value is 1.

In [101]:
type_of_biotransformation = 'hgut'
number_of_steps = 1

run_biotransformer(list_smiles,list_compound_name,type_of_biotransformation,number_of_steps,'results_vm-NAP_BioTransformer.tsv')

======== Running BioTransformer takes approximatively 3-5 compounds per minute ========
     Number of compounds being computed =  57
          BioTransformer candidates for compound n1 = 3
          BioTransformer candidates for compound n2 = 3
          BioTransformer candidates for compound n3 = 10
          BioTransformer candidates for compound n4 = 10
          BioTransformer candidates for compound n5 = 2
          BioTransformer candidates for compound n6 = 4
          BioTransformer candidates for compound n7 = 11
          BioTransformer candidates for compound n8 = 4
          BioTransformer candidates for compound n9 = 8
          BioTransformer candidates for compound n10 = 1
          BioTransformer candidates for compound n11 = 2
          BioTransformer candidates for compound n12 = 2
          BioTransformer candidates for compound n13 = 7
          BioTransformer candidates for compound n14 = 7
          BioTransformer candidates for compound n15 = 8
          BioTran

Download the full BioTransformer results in the left side panel->
['results_vm-NAP_BioTransformer.tsv'](./results_vm-NAP_BioTransformer.tsv).

## Export the results for NAP

See the documentation for custom database in [NAP](https://ccms-ucsd.github.io/GNPSDocumentation/nap/#structure-database) and how to run NAP on GNPS [https://ccms-ucsd.github.io/GNPSDocumentation/nap/#structure-database](https://ccms-ucsd.github.io/GNPSDocumentation/nap/#structure-database).

In [102]:
export_for_NAP('results_vm-NAP_BioTransformer.tsv')

Download the BioTransformer results for NAP in the left side panel->
['results_vm-NAP_BioTransformer_NAP.tsv'](./results_vm-NAP_BioTransformer_NAP.tsv).

## Export the results for SIRIUS

See the documentation to generate the SIRIUS [custom database here](https://boecker-lab.github.io/docs.sirius.github.io/cli-standalone/#custom-database-tool).

In [103]:
export_for_SIRIUS('results_vm-NAP_BioTransformer.tsv')

Download the BioTransformer results for NAP in the left side panel->
['results_vm-NAP_BioTransformer_SIRIUS.tsv'](./results_vm-NAP_BioTransformer_SIRIUS.tsv).